# RN-A — Alineación astrométrica DP1 en 47 Tuc (Rubin×Gaia)

**Propósito.** Reproducir la figura y métricas de separaciones angulares para la RN-A a partir del parquet mínimo.

**Entradas.** `data/47tuc_dp1/rnA_matched_minimal.parquet`

**Salida.** `notebooks/47tuc/figs/rnA_hist_sep.png`

In [ ]:
import os, sys, json
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
print('Versions: pandas', pd.__version__)

In [ ]:
parquet_min = 'data/47tuc_dp1/rnA_matched_minimal.parquet'
fallback = 'data/47tuc_dp1/matched_dp1_gaia.parquet'
if os.path.exists(parquet_min):
    df = pd.read_parquet(parquet_min)
else:
    assert os.path.exists(fallback), 'No se encontró parquet mínimo ni emparejado completo.'
    df = pd.read_parquet(fallback)
    # Intento de renombrado si vienen columnas con otros nombres
    if 'separation_arcsec' not in df.columns:
        for c in df.columns:
            if c.lower().startswith('sep') and 'arcsec' in c.lower():
                df = df.rename(columns={c:'separation_arcsec'})
                break
    for need in ['objectId','coord_ra','coord_dec','source_id','ra_gaia','dec_gaia','separation_arcsec']:
        if need not in df.columns:
            print('ADVERTENCIA: falta columna', need)
df = df.drop_duplicates('objectId').drop_duplicates('source_id')
df = df.query('separation_arcsec <= 2.0')
sep = df['separation_arcsec'].to_numpy()
p50, p68, p95 = np.percentile(sep, [50,68,95])
N = len(df)
print({'N':N,'P50":{:.3f},' 'P68':{:.3f},' 'P95':{:.3f}}.format(p50,p68,p95))

In [ ]:
plt.figure(figsize=(7,5))
bins = np.arange(0, 0.31, 0.01)
plt.hist(sep, bins=bins, color='#1f77b4', ec='white')
plt.axvline(p50, ls='--', lw=2, color='tab:green', label=f'Mediana ({p50:.3f}″)')
plt.axvline(p95, ls=':', lw=2, color='tab:red',   label=f'P95 ({p95:.3f}″)')
plt.xlabel('Separación (arcsec)'); plt.ylabel('Número de pares emparejados')
plt.title('Rubin×Gaia en 47 Tuc — DP1 (ComCam)')
plt.legend(); plt.tight_layout()
outfig = 'notebooks/47tuc/figs/rnA_hist_sep.png'
plt.savefig(outfig, dpi=200)
print('Figura guardada ->', outfig)

**Métricas (esperadas):** N≈1113; P50≈0.051″; P95≈0.116″.

**Control (no figura):** *offset-match* plano y propagación PM Gaia→época DP1 dan sesgos <0.02″ en la mediana.